<a href="https://colab.research.google.com/github/DamodaraBarbosa/eleicoes_ceara_2022/blob/main/05_data_analysis_eleicoes_ceara_2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importando as bibliotecas

In [46]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

# 1. Carregando os dados no *dataframe*

In [2]:
votacao_ce = pd.read_csv(
    '/content/drive/MyDrive/Datasets/votacao_secao_2022_CE_tratado.csv',
    sep= ',', encoding= 'latin-1'
)

In [3]:
votacao_ce.head(3)

,nm_municipio,nr_zona,nr_secao,ds_cargo,nr_votavel,nm_votavel,qt_votos
0,FORTALEZA,94,6,DEPUTADO FEDERAL,4444,HEITOR RODRIGO PEREIRA FREIRE,4
1,RUSSAS,9,266,DEPUTADO FEDERAL,4444,HEITOR RODRIGO PEREIRA FREIRE,13
2,PARAIPABA,109,119,DEPUTADO FEDERAL,4444,HEITOR RODRIGO PEREIRA FREIRE,3


# 2. Informações do *dataframe*

## 2.1 Dimensões do *dataframe*

Definir as dimensões de um *dataframe* é indicar o número de colunas e linhas do mesmo. 

In [4]:
f'O dataframe contém {votacao_ce.shape[1]} colunas e {votacao_ce.shape[0]} linhas.'

'O dataframe contém 7 colunas e 2509903 linhas.'

## 2.2 Infos do *dataframe*

In [5]:
votacao_ce.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2509903 entries, 0 to 2509902
Data columns (total 7 columns):
 #   Column        Dtype 
---  ------        ----- 
 0   nm_municipio  object
 1   nr_zona       int64 
 2   nr_secao      int64 
 3   ds_cargo      object
 4   nr_votavel    int64 
 5   nm_votavel    object
 6   qt_votos      int64 
dtypes: int64(4), object(3)
memory usage: 134.0+ MB


## 2.3 Dados faltantes

É importante identificar a presença de dados faltantes ("not a number" ou `NaN`).

In [6]:
votacao_ce.isna().sum()

nm_municipio    0
nr_zona         0
nr_secao        0
ds_cargo        0
nr_votavel      0
nm_votavel      0
qt_votos        0
dtype: int64

Não há em nenhuma das colunas do conjunto de dados a presença de dados faltantes.

## 2.4 Presença de dados duplicados

Para integridade dos dados também é importante detectar se há dados duplicados. Para isso:

In [7]:
votacao_ce.loc[votacao_ce.duplicated()]

,nm_municipio,nr_zona,nr_secao,ds_cargo,nr_votavel,nm_votavel,qt_votos


Não há dados duplicados no conjunto de dados.

# 3. Tratamento dos dados

In [8]:
votacao_ce['nr_zona'] = votacao_ce['nr_zona'].astype(str)
votacao_ce['nr_secao'] = votacao_ce['nr_secao'].astype(str)

In [9]:
votacao_ce.dtypes

nm_municipio    object
nr_zona         object
nr_secao        object
ds_cargo        object
nr_votavel       int64
nm_votavel      object
qt_votos         int64
dtype: object

# 4. Análise exploratória dos dados

## 4.1 Coluna `nm_municipio`

### 4.1.1 Número de municípios cearenses

In [10]:
f"Há {votacao_ce['nm_municipio'].nunique()} diferentes municípios cearenses na base de dados."

'Há 184 diferentes municípios cearenses na base de dados.'

### 4.1.2 Frequência dos municípios na base de dados

Por serem 184 municípios na base de dados, é adequado reduzir o escopo da análise. Opta-se por analisar apenas os 20 municípios com maiores e menores frequências. 

* Tabela com a frequência absoluta e percentual:

In [11]:
pd.DataFrame(
    {'Frequência absoluta': votacao_ce.value_counts('nm_municipio', ascending= False).head(20),
    'Percentual': round(votacao_ce.value_counts('nm_municipio', ascending= False, normalize= True).head(20) * 100, 2)}
)

,Frequência absoluta,Percentual
nm_municipio,,
FORTALEZA,951899,37.93
CAUCAIA,102428,4.08
JUAZEIRO DO NORTE,73565,2.93
MARACANAÚ,70845,2.82
SOBRAL,48304,1.92
CRATO,33408,1.33
ITAPIPOCA,31179,1.24
MARANGUAPE,29749,1.19
QUIXADÁ,22474,0.90


* Análise gráfica da distribuição dos municípios cearenses:

In [12]:
fig = px.bar(
    x= votacao_ce.value_counts('nm_municipio', ascending= True).tail(20).values,
    y= votacao_ce.value_counts('nm_municipio', ascending= True).tail(20).keys(),
    title= 'Distribuição de frequência dos 20 municípios com maior distribuição de frequência na base de dados'
)
fig.update_layout(title= {'x': 0.5}, yaxis_title= 'Municípios', xaxis_title= 'Frequência')
fig.show()

In [13]:
fig = px.bar(
    x= votacao_ce.value_counts('nm_municipio', ascending= False).tail(20).values,
    y= votacao_ce.value_counts('nm_municipio', ascending= False).tail(20).keys(),
    title= 'Distribuição de frequência dos 20 municípios com menor distribuição de frequência na base de dados',
    color_discrete_sequence= ['red']
)
fig.update_layout(title= {'x': 0.5}, yaxis_title= 'Municípios', xaxis_title= 'Frequência')
fig.show()

O município de Fortaleza, capital do Ceará, é o município com maior distribuição de frequência na base de dados. Sendo mais de nove vezes maior do que Caucaia, o segundo município com mais aparições.

## 4.2 Coluna `nr_zona`

### 4.2.1 Número de zonas no estado do Ceará

In [14]:
f" Ao todo há {votacao_ce['nr_zona'].nunique()} diferentes zonas no estado do Ceará."

' Ao todo há 109 diferentes zonas no estado do Ceará.'

### 3.2.2 Número de zonas por cidade do estado

* Tabela com o número de zonas por município:

In [15]:
zonas_por_municipio = pd.DataFrame({'Nº de zonas': votacao_ce.groupby('nm_municipio')['nr_zona'].nunique().sort_values(ascending= False).head(10)})
zonas_por_municipio

,Nº de zonas
nm_municipio,
FORTALEZA,17
CAUCAIA,3
SOBRAL,2
JUAZEIRO DO NORTE,2
MARACANAÚ,2
ABAIARA,1
PACAJUS,1
MUCAMBO,1
MULUNGU,1


Fortaleza, Caucaia, Sobral, Juazeiro do Norte e Maracanaú são os únicos municípios do Ceará com mais de uma zona eleitora. A capital se destaca com 17 zonas eleitorais. Os demais municípios do estado possuem apenas 1 zona eleitoral.

In [16]:
fig = px.bar(
    data_frame= zonas_por_municipio,
    x= 'Nº de zonas',
    title= 'Os municípios cearenses com mais zonas eleitorais'
)
fig.update_layout(title= {'x': 0.5}, yaxis_title= 'Município', yaxis= dict(autorange= 'reversed')) # inverte a ordem dos dados no eixo y
fig.show()

### 4.2.3 Número de eleitores por zona eleitoral

Selecionam-se apenas os dados referentes a um dos cargos disputados. Os dados são agrupados em `nr_zona` e `ds_cargo`. Somam-se os votos por zona e a tabela é ordenada para que apareçam as vinte zonas com mais votos.

In [17]:
eleitores_por_zona_eleitoral = votacao_ce.query('ds_cargo == "GOVERNADOR"')\
                                 .groupby(['nr_zona', 'ds_cargo'])['qt_votos']\
                                 .sum().sort_values(ascending= False)\
                                 .head(20).to_frame()

Os *index* da tabela agrupada são resetados e a coluna `ds_cargo`, que contém o mesmo cargo, é eliminada.

In [18]:
eleitores_por_zona_eleitoral = eleitores_por_zona_eleitoral.reset_index()\
                                                           .drop('ds_cargo', axis= 1)

O nome `qt_votos` para coluna da tabela não faz sentido, pois motra, na realidade, o número de eleitores por zona eleitoral. Portanto, a coluna passa a ser `eleitores`.

In [19]:
eleitores_por_zona_eleitoral = eleitores_por_zona_eleitoral.rename({'qt_votos': 'eleitores'}, axis= 1)

In [20]:
eleitores_por_zona_eleitoral.dtypes

nr_zona      object
eleitores     int64
dtype: object

In [21]:
fig = px.bar(
    data_frame= eleitores_por_zona_eleitoral,
    y= 'nr_zona',
    x= 'eleitores',
    title= 'As 20 zonas eleitorais com mais eleitores no Ceará'
)
fig.update_layout(title= {'x': 0.5}, yaxis_title= 'Nº da zona eleitoral', xaxis_title= 'Quantidade de eleitores', yaxis= dict(autorange= 'reversed'))
fig.show()

#### 4.2.3.1 Municípios em que se encontram as zonas eleitorais com mais eleitores

In [22]:
zonas_eleitorais = eleitores_por_zona_eleitoral['nr_zona']

A coluna `nr_zona` do *dataframe* `eleitores_por_zona_eleitoral` é armazenada numa nova variável. 

Quando os valores são consultados na base de dados, tem-se os municípios:

In [23]:
votacao_ce.query('nr_zona in @zonas_eleitorais')['nm_municipio'].unique()

array(['FORTALEZA', 'ITAPIPOCA', 'IGUATU', 'QUIXELÔ', 'JUAZEIRO DO NORTE',
       'SOBRAL', 'TURURU', 'CEDRO', 'ALCÂNTARAS', 'MERUOCA', 'JARDIM'],
      dtype=object)

## 4.3 Coluna `nr_secao`

### 4.3.1 Número de seções na base de dados

In [24]:
f"Há {votacao_ce['nr_secao'].nunique()} diferentes seções na base de dados."

'Há 938 diferentes seções na base de dados.'

### 4.3.2 Número de seções por cidade do estado

#### 4.3.2.1 As cidades do estado com mais seções eleitorais

As cidades do estado do Ceará com mais seções eleitorais são:


In [25]:
cidades_mais_secoes = pd.DataFrame({'Nº de seções': votacao_ce.groupby('nm_municipio')['nr_secao'].nunique().sort_values(ascending= False).head(20)})
cidades_mais_secoes

,Nº de seções
nm_municipio,
FORTALEZA,937
CAUCAIA,448
SOBRAL,430
JUAZEIRO DO NORTE,398
MARACANAÚ,386
ITAPIPOCA,336
CRATO,309
MARANGUAPE,278
IGUATU,258


In [26]:
fig = px.bar(
    data_frame= cidades_mais_secoes,
    x= 'Nº de seções',
    y= cidades_mais_secoes.index,
    title= 'As vinte cidades com mais seções eleitorais no Ceará' 
)
fig.update_layout(title= {'x': 0.5}, yaxis_title= 'Nome do município', yaxis= dict(autorange= 'reversed'))
fig.show()

#### 4.3.2.2 As cidades do estado com menos seções eleitorais

In [27]:
cidades_menos_secoes = pd.DataFrame({'Nº de seções': votacao_ce.groupby('nm_municipio')['nr_secao'].nunique().sort_values().head(20)})
cidades_menos_secoes

,Nº de seções
nm_municipio,
GRANJEIRO,20
POTIRETAMA,21
SENADOR SÁ,21
ERERÉ,21
GUARAMIRANGA,22
PACUJÁ,22
BAIXIO,23
ARNEIROZ,25
ITAIÇABA,25


In [28]:
fig = px.bar(
    data_frame= cidades_menos_secoes,
    x= 'Nº de seções',
    y= cidades_menos_secoes.index,
    color_discrete_sequence= ['red'],
    title= 'As vinte cidades com menos seções eleitorais no Ceará'
    
)
fig.update_layout(title= {'x': 0.5}, yaxis_title= 'Nome do município', yaxis= dict(autorange= 'reversed'))
fig.show()

### 4.3.3 Número de eleitores por seção eleitoral

Selecionam-se apenas os dados referentes a um dos cargos disputados. Os dados são agrupados em `nr_secao` e `ds_cargo`. Somam-se os votos por seção e a tabela é ordenada para que apareçam as vinte zonas com mais votos.

In [29]:
eleitores_por_secao_eleitoral = votacao_ce.query('ds_cargo == "GOVERNADOR"')\
                                  .groupby(['nr_secao', 'ds_cargo'])['qt_votos']\
                                  .sum().sort_values(ascending= False)\
                                  .head(20).to_frame()

Os *index* da tabela agrupada são resetados e a coluna `ds_cargo`, que contém o mesmo cargo, é eliminada.

In [30]:
eleitores_por_secao_eleitoral = eleitores_por_secao_eleitoral.reset_index()\
                                                             .drop('ds_cargo', axis= 1)

O nome `qt_votos` para coluna da tabela não faz sentido, pois motra, na realidade, o número de eleitores por zona eleitoral. Portanto, a coluna passa a ser `eleitores`.

In [31]:
eleitores_por_secao_eleitoral = eleitores_por_secao_eleitoral.rename({'qt_votos': 'eleitores'}, axis= 1)

In [32]:
fig = px.bar(
    data_frame= eleitores_por_secao_eleitoral,
    y= 'nr_secao',
    x= 'eleitores',
    title= 'As seções com mais eleitores no estado do Ceará'
)
fig.update_layout(title= {'x': 0.5}, yaxis_title= 'Nº da seção', xaxis_title= 'Quantidade de eleitores', yaxis= dict(autorange= 'reversed'))
fig.show()

#### 4.3.3.1 Municípios em que se encontram as seções eleitorais com mais eleitores

In [33]:
secoes_eleitorais = eleitores_por_secao_eleitoral['nr_secao']

Quando os valores são consultados na base de dados, tem-se os municípios:

In [34]:
votacao_ce.query('nr_secao in @secoes_eleitorais')['nm_municipio'].unique()

array(['ITAPIPOCA', 'PEDRA BRANCA', 'TIANGUÁ', 'MARANGUAPE', 'FORTALEZA',
       'QUIXERAMOBIM', 'LAVRAS DA MANGABEIRA', 'ANTONINA DO NORTE',
       'CAMOCIM', 'MISSÃO VELHA', 'AQUIRAZ', 'UBAJARA', 'JAGUARETAMA',
       'IGUATU', 'ICÓ', 'BREJO SANTO', 'CASCAVEL', 'MASSAPÊ',
       'ALCÂNTARAS', 'MARACANAÚ', 'CARIRÉ', 'SOBRAL', 'IPU',
       'JUAZEIRO DO NORTE', 'COREAÚ', 'TAUÁ', 'BARBALHA', 'ALTANEIRA',
       'BEBERIBE', 'BAIXIO', 'IBIAPINA', 'PACAJUS', 'CAUCAIA', 'ARARIPE',
       'APUIARÉS', 'GUARACIABA DO NORTE', 'ITAPAJÉ', 'CRATO', 'CARIDADE',
       'PARACURU', 'MILAGRES', 'ARACOIABA', 'MORADA NOVA', 'CAMPOS SALES',
       'BATURITÉ', 'IBARETAMA', 'SANTANA DO CARIRI', 'ARACATI',
       'IPUEIRAS', 'IPAUMIRIM', 'RERIUTABA', 'PACATUBA',
       'SÃO GONÇALO DO AMARANTE', 'NOVA RUSSAS', 'TAMBORIL', 'AMONTADA',
       'QUITERIANÓPOLIS', 'TRAIRI', 'ACARAPE', 'CRUZ', 'QUIXADÁ',
       'RUSSAS', 'ERERÉ', 'LIMOEIRO DO NORTE', 'REDENÇÃO', 'AIUABA',
       'CRATEÚS', 'JAGUARUANA', 'ARATUBA'

## 4.4 Coluna `ds_cargo`

Os cargos eleitorais presentes na base de dados são:

In [35]:
votacao_ce['ds_cargo'].unique()

array(['DEPUTADO FEDERAL', 'DEPUTADO ESTADUAL', 'SENADOR', 'GOVERNADOR'],
      dtype=object)

## 4.5 Coluna `nm_votavel`

O número de candidatos, independente dos cargos, em toda a base de dados:

In [36]:
f"A base de dados contém {votacao_ce['nm_votavel'].nunique()} candidatos diferentes."

'A base de dados contém 928 candidatos diferentes.'

# 5. Análise eleitoral

## 5.1 Cargo governador

### 5.1.1 Número de votos por candidato ao governo do estado do Ceará

In [37]:
votos_governador = votacao_ce.query('ds_cargo == "GOVERNADOR"').groupby('nm_votavel')['qt_votos'].sum().sort_values(ascending= False).to_frame()
votos_governador

,qt_votos
nm_votavel,
ELMANO DE FREITAS DA COSTA,2808300
WAGNER SOUSA GOMES,1649213
ROBERTO CLÁUDIO RODRIGUES BEZERRA,734976
VOTO NULO,249099
VOTO BRANCO,175459
FRANCISCO RAIMUNDO MALTA DE ARAÚJO,3015
SERLEY DE SOUSA LEAL,1881
JOSÉ BATISTA NETO,1507


In [38]:
fig = px.bar(
    data_frame= votos_governador,
    y= votos_governador.index,
    x= 'qt_votos',
    color= votos_governador.index,
    title= 'Votos totais por candidato ao cargo de governador do Ceará',
)
fig.update_layout(title= {'x': 0.5}, yaxis_title= 'Candidatos', xaxis_title= 'Quantidade de votos', legend_title= 'Candidatos')
fig.show()

### 5.1.2 Porcentagem de votos totais e válidos ao governo do estado

O número de votos totais de votos para o cargo de governador:

In [39]:
numero_votos_totais = votacao_ce.query('ds_cargo == "GOVERNADOR"')['qt_votos'].sum()
print(f'Votos totais: {numero_votos_totais}')

Votos totais: 5623450


E para determinar o número de votos válidos basta excluir os votos nulos e brancos:

In [40]:
votos_validos = votacao_ce.query('nm_votavel != "VOTO NULO" and nm_votavel != "VOTO BRANCO"')

In [41]:
numero_votos_validos = votos_validos.query('ds_cargo == "GOVERNADOR"')['qt_votos'].sum()
print(f'Votos válidos: {numero_votos_validos}')

Votos válidos: 5198892


A porcentagem é calculada e mostrada na tabela abaixo:

In [42]:
porcentagem_votos_governadores = pd.DataFrame(
    {'Votos válidos':round((votos_validos.query('ds_cargo == "GOVERNADOR"').groupby('nm_votavel')['qt_votos'].sum()/numero_votos_validos) * 100, 2),
     'Votos totais': round((votacao_ce.query('ds_cargo == "GOVERNADOR"').groupby('nm_votavel')['qt_votos'].sum()/numero_votos_totais) * 100, 2)}
)

In [43]:
porcentagem_votos_governadores = porcentagem_votos_governadores.transpose()
porcentagem_votos_governadores

nm_votavel,ELMANO DE FREITAS DA COSTA,FRANCISCO RAIMUNDO MALTA DE ARAÚJO,JOSÉ BATISTA NETO,ROBERTO CLÁUDIO RODRIGUES BEZERRA,SERLEY DE SOUSA LEAL,VOTO BRANCO,VOTO NULO,WAGNER SOUSA GOMES
Votos válidos,54.02,0.06,0.03,14.14,0.04,NaN,NaN,31.72
Votos totais,49.94,0.05,0.03,13.07,0.03,3.12,4.43,29.33


Por fim, o gráfico mostra a comparação entre os votos totais e válidos.

In [51]:
fig = px.bar(
    data_frame= porcentagem_votos_governadores,
    y= porcentagem_votos_governadores.index,
    x= porcentagem_votos_governadores.columns,
    title= 'Porcentagem de votos totais e válidos para o cargo de governador do Ceará'
)
fig.update_layout(title= {'x': 0.5}, yaxis_title= '', xaxis_title= 'Porcentagem dos votos', legend_title= 'Candidatos')
fig.show()

## 5.2 Votos para o cargo de governador por município cearense

### 5.2.1 Contribuição de cada município para o panorama geral de votos no estado

Calcula-se o quanto o número de votos no município representa em relação ao total de votos no estado do Ceará. E depois tal valor é convertido em porcentagem:

In [58]:
votos_municipio_pelo_total = votacao_ce.query('ds_cargo == "GOVERNADOR"').groupby('nm_municipio')['qt_votos'].sum()/numero_votos_totais
# selecionam-se os dados referentes aos votos em governador, são agrupados por município e somados.
votos_porcentagem_municipio = round(votos_municipio_pelo_total * 100, 2) # os valores são convertidos em porcentagem.

In [62]:
votos_porcentagem_municipio = pd.DataFrame(votos_porcentagem_municipio.sort_values('qt_votos', ascending= False)) # ordenam-se os dados do maior para o menor.
votos_porcentagem_municipio.head()

,qt_votos
nm_municipio,
FORTALEZA,27.74
CAUCAIA,3.61
JUAZEIRO DO NORTE,2.75
MARACANAÚ,2.64
SOBRAL,2.23


In [67]:
fig = go.Figure()
fig.add_trace(
    go.Waterfall(x= votos_porcentagem_municipio.index, y= votos_porcentagem_municipio['qt_votos'])
)
fig.update_layout(
    title= 'Contribuição de votos, em porcentagem, dos municípios no panorama geral da votação para governador do Ceará',
    title_x= 0.5, yaxis_title= 'Porcentagem', xaxis_title= 'Municípios'
)
fig.show()

Fortaleza, capital do estado, sozinha corresponde a mais de um quarto dos votos (27,74%) e é seguida por outras cidades importantes como Caucaia (3,61%), Juazeiro do Norte (2,75%), Maracanaú (2,64%) e Sobral (2,23%).

### 5.2.2 Quantidade de municípios em que os candidatos venceram 

O número de cidades em que os candidatos ao governo do Ceará venceram 

In [70]:
votos_candidatos_por_municipio = votacao_ce.query('ds_cargo == "GOVERNADOR"')\
                                           .groupby(['nm_municipio', 'nm_votavel'])\
                                           ['qt_votos'].sum().reset_index()

In [73]:
votos_candidatos_por_municipio = votos_candidatos_por_municipio.sort_values(['nm_municipio', 'qt_votos'], ascending= [True, False])

In [74]:
vencedores_por_municipio = votos_candidatos_por_municipio.groupby('nm_municipio').head(1)

In [79]:
vencedores_por_municipio.head()

,nm_municipio,nm_votavel,qt_votos
0,ABAIARA,ELMANO DE FREITAS DA COSTA,4182
8,ACARAPE,ELMANO DE FREITAS DA COSTA,4652
15,ACARAÚ,ELMANO DE FREITAS DA COSTA,20534
23,ACOPIARA,ELMANO DE FREITAS DA COSTA,18015
31,AIUABA,ELMANO DE FREITAS DA COSTA,4052


In [78]:
fig = px.histogram(
    data_frame= vencedores_por_municipio,
    y= 'nm_votavel', color= 'nm_votavel',
    title= 'Quantidade de municípios cearenses em que cada candidato saiu vencedor'
)
fig.update_layout(title= {'x': 0.5}, yaxis_title= 'Candidatos', xaxis_title= 'Número de cidades', legend_title= 'Candidatos')
fig.show()